In [26]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from datetime import timedelta
from sklearn.metrics import mean_absolute_percentage_error



In [11]:
!git clone https://github.com/balrks/linear-poly-rice-model.git

fatal: destination path 'linear-poly-rice-model' already exists and is not an empty directory.


In [27]:
df = pd.read_csv('/content/linear-poly-rice-model/data_harga_beras_2020_2024.csv')
df


,Date,Semua Provinsi,Aceh,Kota Lhokseumawe
0,2020-01-01,11800.0,10300.0,10000.0
1,2020-01-02,11800.0,10300.0,10000.0
2,2020-01-03,11850.0,10300.0,10000.0
3,2020-01-06,11800.0,10350.0,10000.0
4,2020-01-07,11800.0,10350.0,10000.0
...,...,...,...,...
1213,2024-08-26,15350.0,13900.0,13450.0
1214,2024-08-27,15350.0,13900.0,13450.0
1215,2024-08-28,15250.0,13900.0,13450.0
1216,2024-08-29,15300.0,13900.0,13450.0


In [28]:
# Load the data from the CSV file
df = pd.read_csv('/content/linear-poly-rice-model/data_harga_beras_2020_2024.csv')

# Mengubah kolom 'date' menjadi format datetime
df['Date'] = pd.to_datetime(df['Date'])



In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              1218 non-null   int64  
 1   Semua Provinsi    1218 non-null   float64
 2   Aceh              1218 non-null   float64
 3   Kota Lhokseumawe  1218 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 38.2 KB


In [30]:
# Check missing values
df.isnull().sum()

,0
Date,0
Semua Provinsi,0
Aceh,0
Kota Lhokseumawe,0


In [32]:
# Mengurutkan DataFrame berdasarkan kolom 'date'
df = df.sort_values(by='Date')

# Memberikan urutan berurutan mulai dari 1
df['date'] = range(1, len(df) + 1)

# Mengonversi kolom 'urutan' menjadi tipe data integer (int64)
df['date'] = df['date'].astype('int64')

# Menampilkan hasil
print(df)

      Date  Semua Provinsi     Aceh  Kota Lhokseumawe  date
0        1         11800.0  10300.0           10000.0     1
1        2         11800.0  10300.0           10000.0     2
2        3         11850.0  10300.0           10000.0     3
3        4         11800.0  10350.0           10000.0     4
4        5         11800.0  10350.0           10000.0     5
...    ...             ...      ...               ...   ...
1213  1214         15350.0  13900.0           13450.0  1214
1214  1215         15350.0  13900.0           13450.0  1215
1215  1216         15250.0  13900.0           13450.0  1216
1216  1217         15300.0  13900.0           13450.0  1217
1217  1218         15350.0  14100.0           13450.0  1218

[1218 rows x 5 columns]


In [33]:
df.drop(columns=['date'], inplace=True)

In [35]:
# # 6. Prepare data for regression
X = df['Date'].values.reshape(-1, 1)  # Independent variable
Y = df['Kota Lhokseumawe'].values     # Dependent variable

In [36]:
# Pisahkan data menjadi data pelatihan dan data pengujian
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [37]:
# Fit the Linear Regression model on the train data
linear = LinearRegression()
model_linear = linear.fit(X_train, Y_train)

# # # Print the coefficients and the intercept of the model
print('Nilai intercept (a): ',linear.intercept_)
print('Nilai slope (b): ',linear.coef_)

Nilai intercept (a):  8805.79649552436
Nilai slope (b):  [3.62721857]


In [38]:
# Initialize PolynomialFeatures
poly = PolynomialFeatures()

X_train_poly = poly.fit_transform(X_train)

poly_model = LinearRegression()
poly_model.fit(X_train_poly, Y_train)

# Print the coefficients and the intercept
print('Nilai Intercept:', poly_model.intercept_)
print("Nilai slope:", poly_model.coef_)


Nilai Intercept: 9947.783130437068
Nilai slope: [ 0.         -2.0219701   0.00463403]
